In [ ]:
import iyer_saliency
import sarfa_saliency

import chess 
import chess.uci
import time

from sys import platform as _platform
from collections import defaultdict


from IPython.display import Image, display
import cairosvg
import cv2
import svg_custom.svg_custom as svg_custom 
import matplotlib.pyplot as plt
import numpy as np

import json 
import pandas as pd 
from pandas.io.json import json_normalize 

from collections import defaultdict
import operator
from pymoo.factory import get_performance_indicator

import os
import matplotlib.pyplot as plt

from sklearn import metrics
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score

plt.rcParams['figure.figsize']=(16,16)

In [ ]:
def select_engine():
    handler = chess.uci.InfoHandler()
    if _platform == "linux" or _platform == "linux2":

        engine = chess.uci.popen_engine('engines/stockfish-11-linux/stockfish-11-linux/Linux/stockfish_20011801_x64')

    elif _platform == "darwin":
        engine = chess.uci.popen_engine('engines/stockfish-11-mac/stockfish-11-mac/Mac/stockfish-11-64')

    elif _platform == "win32":
        engine = chess.uci.popen_engine('engines/stockfish-11-win/stockfish-11-win/Windows/stockfish_20011801_32bit.exe') 

    elif _platform == "win64":
        engine = chess.uci.popen_engine('engines/stockfish-11-win/stockfish-11-win/Windows/stockfish_20011801_x64.exe')
    engine.setoption({'MultiPV': 100})
    engine.info_handlers.append(handler)
    return handler,engine

In [ ]:

def get_dict_q_vals(board, legal_moves, eval_time, original_move,engine,handler):
    """
    Function returns Q-values in given board position
    Input:
        board: chess.Board()
        legal_moves: List of legal moves of original state
        eval_time: Search time for stockfish
        original_move: original best move (chess.Move()) 
    Output:
        q_vals_dict: Dictionary containing Q-values and Actions
        bestmove: chess.Move() - Best move in given board position
    """
    
    i = 0
    q_vals_dict = {}
    
    
    set_current_legal_moves = set(board.legal_moves)
    set_original_legal_moves = set(legal_moves)
    intersection_set = set_current_legal_moves.intersection(set_original_legal_moves)

    #print('querying engine with perturbed position')
    engine.position(board)
    evaluation = engine.go(movetime=eval_time)
    #print(evaluation)
    if original_move is None:
        # no initial move supplied
        original_move = evaluation.bestmove
    dict_moves_to_score = defaultdict(int)
    
    for move_id in handler.info['pv'].keys():
        move_string = str(handler.info['pv'][move_id][0])
        move_score = 0
        if handler.info["score"][move_id].cp is None:
            mate_in_moves = handler.info["score"][move_id].mate
            if mate_in_moves > 0:
                # white will win in some number of moves
                move_score = 40
            else:
                # black will win 
                move_score = -40
        else:
            move_score = round(handler.info["score"][move_id].cp/100.0,2)
        dict_moves_to_score[move_string] = move_score
    
    #print(dict_moves_to_score)
    
    #print('Total Legal Moves : ', len(intersection_set))

    for el in legal_moves:
        if el in intersection_set:
            i += 1
            score = dict_moves_to_score[str(el)]
            q_vals_dict[el.uci()] = score
    
    return q_vals_dict, evaluation.bestmove

In [ ]:
def computeSaliency(FEN = 'rnbq1rk1/pp2bppp/4p3/3p3n/3P1B2/3B1N2/PPPNQPPP/R3K2R w KQkq - 0 1',typeSal='sarfa',engine= chess.uci.popen_engine('engines/stockfish-11-linux/stockfish-11-linux/Linux/stockfish_20011801_x64'),handler=None):
    """
    Function returns saliency map for given board position  
    Input:
        FEN : Board position encoded in a FEN  
    Output:
        answer : Saliency for each location on the board
    """

    print("***********************", FEN, "**********************")
    board = chess.Board(FEN)
    evaltime = 6000
    legal_moves = list(board.legal_moves)[:]

    # Q-values for original state
    dict_q_values_before_perturbation, original_move  = get_dict_q_vals(board, legal_moves, evaltime, None,engine,handler)
    #print('original move = ', original_move)
    
    # Saliency for each board location
    answer = {
        'a1' : {'int': chess.A1, 'saliency': -2},
        'a2' : {'int': chess.A2, 'saliency': -2},
        'a3' : {'int': chess.A3, 'saliency': -2},
        'a4' : {'int': chess.A4, 'saliency': -2},
        'a5' : {'int': chess.A5, 'saliency': -2},
        'a6' : {'int': chess.A6, 'saliency': -2},
        'a7' : {'int': chess.A7, 'saliency': -2},
        'a8' : {'int': chess.A8, 'saliency': -2},
        'b1' : {'int': chess.B1, 'saliency': -2},
        'b2' : {'int': chess.B2, 'saliency': -2},
        'b3' : {'int': chess.B3, 'saliency': -2},
        'b4' : {'int': chess.B4, 'saliency': -2},
        'b5' : {'int': chess.B5, 'saliency': -2},
        'b6' : {'int': chess.B6, 'saliency': -2},
        'b7' : {'int': chess.B7, 'saliency': -2},
        'b8' : {'int': chess.B8, 'saliency': -2},
        'c1' : {'int': chess.C1, 'saliency': -2},
        'c2' : {'int': chess.C2, 'saliency': -2},
        'c3' : {'int': chess.C3, 'saliency': -2},
        'c4' : {'int': chess.C4, 'saliency': -2},
        'c5' : {'int': chess.C5, 'saliency': -2},
        'c6' : {'int': chess.C6, 'saliency': -2},
        'c7' : {'int': chess.C7, 'saliency': -2},
        'c8' : {'int': chess.C8, 'saliency': -2},
        'd1' : {'int': chess.D1, 'saliency': -2},
        'd2' : {'int': chess.D2, 'saliency': -2},
        'd3' : {'int': chess.D3, 'saliency': -2},
        'd4' : {'int': chess.D4, 'saliency': -2},
        'd5' : {'int': chess.D5, 'saliency': -2},
        'd6' : {'int': chess.D6, 'saliency': -2},
        'd7' : {'int': chess.D7, 'saliency': -2},
        'd8' : {'int': chess.D8, 'saliency': -2},
        'e1' : {'int': chess.E1, 'saliency': -2},
        'e2' : {'int': chess.E2, 'saliency': -2},
        'e3' : {'int': chess.E3, 'saliency': -2},
        'e4' : {'int': chess.E4, 'saliency': -2},
        'e5' : {'int': chess.E5, 'saliency': -2},
        'e6' : {'int': chess.E6, 'saliency': -2},
        'e7' : {'int': chess.E7, 'saliency': -2},
        'e8' : {'int': chess.E8, 'saliency': -2},
        'f1' : {'int': chess.F1, 'saliency': -2},
        'f2' : {'int': chess.F2, 'saliency': -2},
        'f3' : {'int': chess.F3, 'saliency': -2},
        'f4' : {'int': chess.F4, 'saliency': -2},
        'f5' : {'int': chess.F5, 'saliency': -2},
        'f6' : {'int': chess.F6, 'saliency': -2},
        'f7' : {'int': chess.F7, 'saliency': -2},
        'f8' : {'int': chess.F8, 'saliency': -2},
        'g1' : {'int': chess.G1, 'saliency': -2},
        'g2' : {'int': chess.G2, 'saliency': -2},
        'g3' : {'int': chess.G3, 'saliency': -2},
        'g4' : {'int': chess.G4, 'saliency': -2},
        'g5' : {'int': chess.G5, 'saliency': -2},
        'g6' : {'int': chess.G6, 'saliency': -2},
        'g7' : {'int': chess.G7, 'saliency': -2},
        'g8' : {'int': chess.G8, 'saliency': -2},
        'h1' : {'int': chess.H1, 'saliency': -2},
        'h2' : {'int': chess.H2, 'saliency': -2},
        'h3' : {'int': chess.H3, 'saliency': -2},
        'h4' : {'int': chess.H4, 'saliency': -2},
        'h5' : {'int': chess.H5, 'saliency': -2},
        'h6' : {'int': chess.H6, 'saliency': -2},
        'h7' : {'int': chess.H7, 'saliency': -2},
        'h8' : {'int': chess.H8, 'saliency': -2},

        }
    
    # Iteratively perturb each feature on the board
    # Note : Perturbations should be valid. Code for avoiding those cases.
    
    for square_string in sorted(answer.keys()):
        entry = answer[square_string]
        entry_keys = ['saliency']
        #print('perturbing square = ', square_string)
        # perturb board
        piece_removed = board.remove_piece_at(entry['int'])
        
        if piece_removed is None:
            # square was empty, so proceed without changing anything
            #print('square was empty, so skipped')
            # print(board)
            #print('------------------------------------------')
        
            continue
        
        elif (piece_removed == chess.Piece(6,True) or piece_removed == chess.Piece(6,False)) or board.was_into_check():
            # illegal piece was removed
            #print('illegal piece was removed')
            for key in entry_keys:
                entry[key] = 0
        else:
            # set perturbed state
            engine.position(board)

            # Check if the original move is still valid
            if board.is_legal(original_move):
                # Find the q values 
                dict_q_values_after_perturbation, _ = get_dict_q_vals(board, legal_moves, evaltime, original_move,engine,handler)
                if typeSal=='sarfa':
                    entry['saliency'], entry['dP'], entry['K'], entry['QMaxAnswer'],\
                    entry['actionGapBeforePerturbation'], entry['actionGapAfterPerturbation']\
                         = sarfa_saliency.computeSaliencyUsingSarfa(str(original_move), dict_q_values_before_perturbation, dict_q_values_after_perturbation)
                    #entry['saliency'], entry['dP'], entry['K'] =sarfa_saliency.computeSaliencyUsingSarfa(str(original_move), dict_q_values_before_perturbation, dict_q_values_after_perturbation)
                    
                    dd = defaultdict(list)

                    for d in (dict_q_values_after_perturbation, dict_q_values_before_perturbation): # you can list as many input dicts as you want here
                        for key, value in d.items():
                            dd[key].append(value)
                    entry['Q-Value']=dict(dd)

                else:
                    entry['saliency']=iyer_saliency.computeSaliencyUsingIyer(str(original_move), dict_q_values_before_perturbation, dict_q_values_after_perturbation)
            else:
                # illegal original move in perturbed state, therefore piece removed is probably important 
                # print(board.is_legal(original_move))
                # print(board)
                #print('original move illegal in perturbed state')
                for key in entry_keys:
                    entry[key] = -1
                entry['saliency'] = 1
                #entry['dP']=1
                #entry['K']=1
                
        # undo perturbation
        #print('------------------------------------------')
                
        board.set_piece_at(entry['int'], piece_removed)
        
    #print('BBBOB',answer)
    return answer

In [ ]:
# Cell for displaying board position

# Few utility functions
def svg_to_png(img):
    '''
    Converts given svg image to png
    Input : 
        img : image in .svg format 
    Output :
        svg_custom/board.png
        Display of image
    '''
    with open('svg_custom/board.svg', 'w+') as f:
        f.write(img)
    cairosvg.svg2png(url='svg_custom/board.svg', write_to='svg_custom/board.png')
    display(Image(filename='svg_custom/board.png'))

def display_board(board):
    '''
    Displaying given board
    Input : 
        board : chess.Board
    Output :
        svg_custom/board.png
        Display of image     
    '''
    img = svg_custom.board(board)
    svg_to_png(img)
    
def return_bestmove(board,engine, eval_time = 6000):
    '''
    Returns and displays best move for a given chess position
    Input :
        board : chess.Board
    Output :
        bestmove : chess.Move 
    '''
    engine.position(board)
    bestmove =  engine.go(movetime=eval_time).bestmove
    #print('Best move is', bestmove)
    
    #svg_w_arrow = svg_custom.board(board, arrows = [svg_custom.Arrow(tail =  bestmove.from_square, head = bestmove.to_square, color = '#e6e600')])
    
    #svg_to_png(svg_w_arrow)
    
    return bestmove

def explanation(board,typeS,engine,handler):
    '''
    Generates explanation of the best move for given board position using SARFA
    
    Input :
        board = chess.Board
    Output :
        prints piecewise saliency
        svg_custom/board.png
    '''
    
    bestmove =  engine.go(movetime=6000).bestmove

    # Evaluation of board position for best move
    evaluation= computeSaliency(chess.Board.fen(board),typeS,engine,handler)
    #print('EVALUATIIIION',evaluation)

    return evaluation, bestmove 

def generate_heatmap(evaluation, bestmove):
    """
    Generates heatmap for  saliency evaluation of the best move

    """
    # Laying the saliency map over the board
    heatmap = np.zeros((8, 8))
    for position in evaluation:
        x, y = evaluation[position]['int']//8, evaluation[position]['int'] % 8
        heatmap[x, y] = evaluation[position]['saliency']
    heatmap = np.flipud(heatmap)
    #è una matrice 8X8 dove all'interno ci sono i valori delle saliency per ogni casella    

    #### Saliency map overlaid on board
    svg = svg_custom.board(board, arrows = [svg_custom.Arrow(tail =  bestmove.from_square, head = bestmove.to_square, color = '#e6e600')])

    with open('svg_custom/board.svg', 'w+') as f:
        f.write(svg)
    cairosvg.svg2png(url='svg_custom/board.svg', write_to='svg_custom/board.png')

    # original board as a numpy array
    board_array = cv2.imread('svg_custom/board.png')

    threshold = (100/256)*np.max(heatmap) # percentage threshold. Saliency values above this threshold won't be mapped onto board

    #miserve(evaluation,threshold)
    
    
    # Create bounding boxes with saliency colours for every square on chess board
    for i in range(0, 8, 1):
        for j in range(0, 8, 1):
            ii = 45*i+20
            jj = 45*j+20
            value_of_square =  heatmap[i, j]
            if value_of_square < threshold:
                continue
            for box_i in range(ii, ii+44, 1):
                for box_j in range(jj, jj+44, 1):
                    if box_i > ii+4 and box_i < ii+40 and box_j > jj+4 and box_j < jj+40:
                        continue
                    board_array[box_i, box_j, 0] = 256 - 0.8*256*heatmap[i, j]/(np.max(heatmap) + 1e-10)
                    board_array[box_i, box_j, 1] = 256 - 0.84*256*heatmap[i, j]/(np.max(heatmap) + 1e-10)
                    board_array[box_i, box_j, 2] = 256 - 0.19*256*heatmap[i, j]/(np.max(heatmap) + 1e-10)

    cv2.imwrite("svg_custom/board.png", board_array)
    #print('------------===============------------===============\n\n')
    display(Image('svg_custom/board.png'))
    #print('\n\n------------===============------------===============')

In [ ]:
def saliency(evaluation,bestmove):
    # Laying the saliency map over the board
    heatmap = np.zeros((8, 8))
    for position in evaluation:
        x, y = evaluation[position]['int']//8, evaluation[position]['int'] % 8
        heatmap[x, y] = evaluation[position]['saliency']
    heatmap = np.flipud(heatmap)
    #è una matrice 8X8 dove all'interno ci sono i valori delle saliency per ogni casella    
    
    threshold = (100/256)*np.max(heatmap) # percentage threshold. Saliency values above this threshold won't be mapped onto board

    dict_sarfa={}
    for k in evaluation.keys():
        if evaluation[k]['saliency'] >= threshold:
            dict_sarfa[k]=evaluation[k]['saliency']
    return dict_sarfa

In [ ]:
def gameOne(args):
    FEN,temp,index = args
    handler,engine=select_engine()

    board = chess.Board(FEN)
    best_for_eng=return_bestmove(board,engine)


    for t in temp:
        evaluation, bestmove = explanation(board,t,engine,handler)
        dict_sal= saliency(evaluation,bestmove)
        max_key = max(dict_sal, key=dict_sal.get)
        df_explode=df_sarfa.explode('saliencyGroundTruth')
        file_name=f'file/game_{index}.json'
        if t=='sarfa':
            d =  {"FEN":FEN,
                    "BEST_MOVE":chess.Move.uci(best_for_eng),   
                     "EXPERT":df_sarfa.loc[(df_sarfa['fen']==FEN)]['saliencyGroundTruth'].tolist()[0],
                     "SARFA":[{key:value} for key, value in dict_sal.items()],
                     "Q-VALUE":[{p_id:p_info[key]} for p_id, p_info in evaluation.items() for key in p_info if key=='Q-Value'],
                     "K":[{p_id:p_info[key]} for p_id, p_info in evaluation.items() for key in p_info if key=='K'],
                     "DP":[{p_id:p_info[key]} for p_id, p_info in evaluation.items() for key in p_info if key=='dP'],
                    'EvalSarfa':evaluation

                }
            with open(file_name, 'w') as json_file:
                json.dump(d, json_file)
        else:
            c= {"IYER":[{key:value} for key, value in dict_sal.items()],
               'EvalIyer':evaluation}
            with open(file_name, "r+") as file:
                data = json.load(file)
                data.update(c)
                file.seek(0)
                json.dump(data, file)


## Play games and generate files

In [ ]:
from multiprocessing import Pool, Value
from time import sleep

if __name__ == '__main__':
    #inputs = os.listdir(some_directory)

    #
    # initialize a cross-process counter and the input lists
    #
    #load json object
    with open('chess_saliency_databases/chess-saliency-dataset-v1.json') as f:
        d = json.load(f)

    df_sarfa = pd.json_normalize(d['puzzles'])
    args=[]
    number=1
    for i in range(0,number):
        args.append((df_sarfa['fen'][i],['sarfa','iyer'],i))
    with Pool(64) as p:
        p.map(gameOne, args)



        


## Generate Pareto

In [ ]:
test

In [ ]:
def identify_pareto(test,index):
    fig, ax = plt.subplots()

    data = list(test.values())
    scores = np.array(data)
    # Count number of items
    population_size = scores.shape[0]
    # Create a NumPy index for scores on the pareto front (zero indexed)
    population_ids = np.arange(population_size)
    # Create a starting list of items on the Pareto front
    # All items start off as being labelled as on the Parteo front
    pareto_front = np.ones(population_size, dtype=bool)
    # Loop through each item. This will then be compared with all other items
    for i in range(population_size):
        # Loop through all other items
        for j in range(population_size):
            # Check if our 'i' pint is dominated by out 'j' point
            if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                # j dominates i. Label 'i' point as not on Pareto front
                pareto_front[i] = 0
                # Stop further comparisons with 'i' (no more comparisons needed)
                break
    # Return ids of scenarios on pareto front
    pareto =population_ids[pareto_front]
    pareto_front = scores[pareto]
    pareto_front_df = pd.DataFrame(pareto_front)
    pareto_front_df.sort_values(0, inplace=True)
    pareto_front = pareto_front_df.values

    x_all = scores[:, 0]
    y_all = scores[:, 1]
    x_pareto = pareto_front[:, 0]
    y_pareto = pareto_front[:, 1]

    ax.scatter(x_all, y_all)
    ax.plot(x_pareto, y_pareto, color='r')
    ax.set_xlabel("K")
    ax.set_ylabel("DP")
    file_name=f'graphs/game_{index}.png'
    plt.savefig(file_name)
    #plt.show()

In [ ]:
def plot_pareto_frontier(test,d_Sarfa,d_Iyer,d_Expert,index,maxX=True, maxY=True):
    fig, ax = plt.subplots()

    data = list(test.values())
    key=list(test.keys())
    scores = np.array(data)
    keys=np.array(key)
    # Count number of items
    population_size = scores.shape[0]
    try:            
        max_sarfa=[k for k,v in sorted(d_Sarfa.items(), key=lambda item: item[1], reverse=True)][1]
    except:
        max_sarfa=max(d_Sarfa.items(), key=operator.itemgetter(1))[0]
    max_iyer=max(d_Iyer.items(), key=operator.itemgetter(1))[0]
    
    # Count number of items
    population_size = len(test)
    # Create a NumPy index for scores on the pareto front (zero indexed)
    population_ids = np.arange(population_size)
    # Create a starting list of items on the Pareto front
    # All items start off as being labelled as on the Parteo front
    pareto_front = np.ones(population_size, dtype=bool)
    

    # Loop through each item. This will then be compared with all other items
    for i in range(population_size):
        # Loop through all other items
        for j in range(population_size):
            # Check if our 'i' pint is dominated by out 'j' point
            if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                # j dominates i. Label 'i' point as not on Pareto front
                pareto_front[i] = 0
                # Stop further comparisons with 'i' (no more comparisons needed)
                break
    # Return ids of scenarios on pareto front
    pareto =population_ids[pareto_front]
    pareto_front = scores[pareto]
    pareto_label=keys[pareto]

    pareto_front_df = pd.DataFrame(pareto_front)
    pareto_front_df.sort_values(0, inplace=True)
    pareto_front = pareto_front_df.values

    
    count = sum(f in d_Expert for f in pareto_label)
   
    #distanza media expert


    #print('Number of points indicated by experts who are also pareto optimal: ', count)
    gd = get_performance_indicator("gd", np.array(pareto_front))
    #print("GD", gd.calc(np.array(ss)))
    not_optimal=list(set(d_Expert) - set(pareto_label))
    cc = f'Number of points indicated by experts who are also pareto optimal: {count}'
    gg= f'Generational Distance: {gd.calc(np.array(scores))}'

    
    Xs,Ys=zip(*test.values())
    labels = test.keys()   
    plt.scatter(Xs,Ys)
    x_pareto = pareto_front[:, 0]
    y_pareto = pareto_front[:, 1]

    ax.scatter(Xs, Ys)
    ax.plot(x_pareto, y_pareto, color='r')
    
    ax.set_xlabel("K")
    ax.set_ylabel("DP")
    for label, x, y in zip(labels, Xs, Ys):
        plt.annotate(label, xy = (x, y))
        if label in d_Expert:
            plt.scatter(x, y,color='b',edgecolors='y',linewidth=2,label='Expert')
        if label==max_sarfa:
            plt.scatter(x, y,color='black',s=10**2,label='Sarfa')
        if label==max_iyer:
            plt.scatter(x, y,marker='*',s=8**2,color='green',label='Iyer')
    ax.text(0.1,0.6,cc)
    ax.text(0.1,0.4,gg)
    ax.set_xlabel("K")
    ax.set_ylabel("DP")
    ax.legend()
    file_name=f'graphs/game_{index}.png'
    plt.savefig(file_name)
    #plt.show()


## Generate ROC

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
def makeRoc(d_Expert,data):    
    # Saliency for each board location
    answer = {
        'a1' : 0,
        'a2' : 0,
        'a3' : 0,
        'a4' : 0,
        'a5' : 0,
        'a6' : 0,
        'a7' : 0,
        'a8' : 0,
        'b1' : 0,
        'b2' : 0,
        'b3' : 0,
        'b4' : 0,
        'b5' : 0,
        'b6' : 0,
        'b7' : 0,
        'b8' : 0,
        'c1' : 0,
        'c2' : 0,
        'c3' : 0,
        'c4' : 0,
        'c5' : 0,
        'c6' : 0,
        'c7' : 0,
        'c8' : 0,
        'd1' : 0,
        'd2' : 0,
        'd3' : 0,
        'd4' : 0,
        'd5' : 0,
        'd6' : 0,
        'd7' : 0,
        'd8' : 0,
        'e1' : 0,
        'e2' : 0,
        'e3' : 0,
        'e4' : 0,
        'e5' : 0,
        'e6' : 0,
        'e7' : 0,
        'e8' : 0,
        'f1' : 0,
        'f2' : 0,
        'f3' : 0,
        'f4' : 0,
        'f5' : 0,
        'f6' : 0,
        'f7' : 0,
        'f8' : 0,
        'g1' : 0,
        'g2' : 0,
        'g3' : 0,
        'g4' : 0,
        'g5' : 0,
        'g6' : 0,
        'g7' : 0,
        'g8' : 0,
        'h1' : 0,
        'h2' : 0,
        'h3' : 0,
        'h4' : 0,
        'h5' : 0,
        'h6' : 0,
        'h7' : 0,
        'h8' : 0,

        }
    answer.update((k, 1) for k, v in answer.items() if k in d_Expert)
    y=np.array(list(answer.values()))
    probs_i=[p_info[key] for p_id, p_info in data['EvalIyer'].items() for key in p_info if key=='saliency']
    probs_i=NormalizeData(probs_i)
    probs_s=[p_info[key] for p_id, p_info in data['EvalSarfa'].items() for key in p_info if key=='saliency']
    probs_s=NormalizeData(probs_s)
    return y,probs_i,probs_s
    


In [ ]:
all_expert=[]
all_sarfa=[]
all_iyer=[]
number=100
with open('chess_saliency_databases/chess-saliency-dataset-v1.json') as f:
    d = json.load(f)

df_sarfa = pd.json_normalize(d['puzzles'])
for i in range(0,number):
   
    with open(f'file/game_{i}.json') as json_data:
        data = json.load(json_data)
    data_graph=pd.DataFrame.from_dict(data, orient='index').T.set_index('FEN')  
    dd = defaultdict(list)
    d_K=dict((key,d[key]) for d in data['K'] for key in d)
    d_DP=dict((key,d[key]) for d in data['DP'] for key in d)
    d_Sarfa=dict((key,d[key]) for d in data['SARFA'] for key in d)
    d_Iyer=dict((key,d[key]) for d in data['IYER'] for key in d)
    d_Expert=data['EXPERT']
    #y,probs_i,prob_s=makeRoc(d_Expert,data)
    #all_expert.append(y)
    #all_iyer.append(probs_i)
    #all_sarfa.append(prob_s)
    
    


    for d in (d_K, d_DP): 
        for key, value in d.items():
            dd[key].append(value)

    test=dict(dd)
    plot_pareto_frontier(test,d_Sarfa,d_Iyer,d_Expert,i)

In [ ]:
sorted(test.items(), key=lambda x: x[1], reverse=True)


In [ ]:
all_expert=np.array(all_expert)
all_expert = [item for sublist in all_expert for item in sublist]
all_iyer=np.array(all_iyer)
all_iyer = [item for sublist in all_iyer for item in sublist]
all_sarfa=np.array(all_sarfa)
all_sarfa = [item for sublist in all_sarfa for item in sublist]

In [ ]:
result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

for e in [all_sarfa,all_iyer]:
    fpr, tpr, thresholds = metrics.roc_curve(all_expert, e)
    auc=roc_auc_score(all_expert, e)
    result_table = result_table.append({'classifiers':'s',
                                    'fpr':fpr, 
                                    'tpr':tpr, 
                                    'auc':auc}, ignore_index=True)



In [ ]:
fig = plt.figure(figsize=(8,6))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'], 
             result_table.loc[i]['tpr'], 
             label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("Flase Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()